In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # Adjust the settings in the next cell if necessary

# In[3]:


import pytesseract

# COMMENT IF NOT USING WINDOWS (i.e., add # before both lines)
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract"
TESSDATA_PREFIX = r"C:\Program Files\Tesseract-OCR"

# Change to False if you always want your actual main stat value
assume_max_lv_gear = True

# Screenshot path
sh_path = 'screenshots/'


# # Do not change anything below here

# In[4]:


from PIL import Image
from pytesseract import image_to_string

def process(k,img):
    if  k == 'plus' or k == 'level':
        # Since level and plus are read off the gear icon, there is a lot of noise
        # So we need an iterative process to try to extract a value
        thresh = cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU
        low = 81
        proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_RGB2GRAY), low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
        data = image_to_string(Image.fromarray(proc), lang='Hangul', config='--psm 7').replace('+b','6').replace('>','0')
        if not any(i.isdigit() for i in data):
            low = 100
            proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_RGB2GRAY), low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
            data = image_to_string(Image.fromarray(proc), lang='Hangul', config='--psm 7').replace('+b','6').replace('>','0')
            if not any(i.isdigit() for i in data):
                low = 125
                proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_RGB2GRAY), low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
                data = image_to_string(Image.fromarray(proc), lang='Hangul', config='--psm 7').replace('+b','6').replace('>','0')
                if not any(i.isdigit() for i in data): data = str(0)
    else:
        thresh = cv2.THRESH_BINARY_INV
        low = 70
        proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_BGR2GRAY), low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
        data = image_to_string(Image.fromarray(proc), lang='Hangul', config='--psm 6')
        
        if k == 'type':
            for i in data.split('\n'):
                if '전설' in i or '영웅' in i or '고급' in i or '희귀' in i or '일반' in i:
                    data = i

    return data

def stat_converter(stat):
    stat = stat.strip().replace(' ','')
    result = ''
    if '공격력' in stat.lower():
        result = 'Atk'
        if '%' in stat: result += 'P'
    if '생명력' in stat.lower():
        result = 'HP'
        if '%' in stat: result += 'P'
    if '방어력' in stat.lower():
        result = 'Def'
        if '%' in stat: result += 'P'
    if '속도' in stat.lower(): result = 'Spd'
    if '치명확률' in stat.lower(): result = 'Cchance'
    if '치명피해' in stat.lower(): result = 'CDmg'
    if '효과적중' in stat.lower(): result = 'Eff'
    if '효과저항' in stat.lower(): result = 'Res'
    return result
  

def digit_filter(val):
    if not val: return 0
    elif val == '.': return 0
    else: return int(''.join(filter(str.isdigit,val)))

def char_filter(val):
    return ''.join(filter(str.isalpha,val)).capitalize()
  
def max_stat(data,item):
    stat = stat_converter(data)
    val = digit_filter(data) # Begin by setting val = actual value
    if item['ability'] < 15: # Only change stats on items where they need to be increased
        if item['level'] in range(58,73):
            if stat == 'CChance': val = 45
            elif stat == 'CDmg': val = 55
            elif stat == 'Spd': val = 35
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 50
            elif stat == 'HP': val = 2295 # Not exactly right, finer-grained scaling
            elif stat == 'Def': val = 250 # Not exactly right, finer-grained scaling
            elif stat == 'Atk': val = 425 # Not exactly right, finer-grained scaling
        elif item['level'] in range(74,86):
            if stat == 'CChance': val = 55
            elif stat == 'CDmg': val = 65
            elif stat == 'Spd': val = 40
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 60
            elif stat == 'HP': val = 2700 # Not exactly right, finer-grained scaling
            elif stat == 'Def': val = 300 # Not exactly right, finer-grained scaling
            elif stat == 'Atk': val = 500 # Not exactly right, finer-grained scaling
        elif item['level'] in range(87,89):
            if stat == 'CChance': val = 60
            elif stat == 'CDmg': val = 70
            elif stat == 'Spd': val = 45
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 65
            elif stat == 'HP': val = 2765
            elif stat == 'Def': val = 310
            elif stat == 'Atk': val = 515
    return val


# In[2]:


from glob import glob
from string import ascii_lowercase, digits
import random
import json
import cv2

# Format the json for the optimizer
export = {'processVersion':'1','heroes':[],'items':[]}
temp_list = []

print('Beginning export process...')

# Iterate through the screenshots
filenames = glob(sh_path+'*.png')
filenames.extend(glob(sh_path+'*.jpg'))

for n,name in enumerate(filenames):
    
    # The height of the item box changes depending on the length of the item and set descriptions,
    # we have to crop the top and bottom info separately in order to ensure the OCR boxes within these areas
    # remain in fixed locations. we then process the top and bottom info independently.
    
    img = cv2.imread(name)
    
    # Top box
    temp_top = cv2.imread('e7/top.jpg',0)
    a, b, c, max_loc = cv2.minMaxLoc(cv2.matchTemplate(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), temp_top, cv2.TM_CCOEFF_NORMED))
    top_box = img[max_loc[1]:max_loc[1]+160,740:1190] # Fixed width, then crop 160 pixels from top triangle

    import matplotlib.pyplot as plt
    # top_box.save('~/tmp/top_box.png')

    # Bottom box
    temp_bot = cv2.imread('e7/bottom.jpg',0)
    a, b, c, max_loc = cv2.minMaxLoc(cv2.matchTemplate(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), temp_bot, cv2.TM_CCOEFF_NORMED))
    bottom_box = img[max_loc[1]+25:max_loc[1]+360,740:1190] # Fixed width, shift down 25 from divider, then crop 335 pixels deep

    # bottom_box.save('~/tmp/bottom_box.png')

    # Setup item dictionary
    item = {'locked':False,'efficiency':0}

    # Process top image
    top_coords = {'type': [[20,70],[170,432]],
                  'level': [[19,44],[37,68]],
                  'plus': [[11,34],[139,168]]}
    
    print('currently processing',name)

    rarity_cord = {'전설' : 'Epic', '영웅' : 'Heroic', '희귀' : 'Rare', '고급' : 'Good'}
    slot_cords = {'목걸이' : 'Necklace', '반지' : 'Ring', '신발' : 'Boots', '무기' : 'Weapon', '투구' : 'Helmet', '갑옷' : 'Armor'}
    
    for k in top_coords.keys():
        data = process(k,top_box[top_coords[k][0][0]:top_coords[k][0][1],top_coords[k][1][0]:top_coords[k][1][1]])
        if k == 'type':
            item['rarity'] = rarity_cord[data.split(' ')[0]]
            #item['rarity'] = char_filter(data.split(' ')[0])
            item['slot'] = slot_cords[data.split(' ')[1].split('\n')[0]]
            #item['slot'] = char_filter(data.split(' ')[1].split('\n')[0])
        if k == 'level':
            item['level'] = digit_filter(data.replace('S','5').replace('B','8').replace('a','8'))
        if k == 'plus':
            item['ability'] = digit_filter(data.replace('S','5').replace('B','8').replace('a','8'))

    print(name, "shows lvl", item['level'],"and enhance lv",item['ability'])
    
    # Process bottom image
    bot_coords = {'main': [[8,70],[65,435]],
                  'subs': [[98,255],[25,435]],
                  'set': [[280,340],[76,435]]}
    
    for k in bot_coords.keys():
        data = process(k,bottom_box[bot_coords[k][0][0]:bot_coords[k][0][1],bot_coords[k][1][0]:bot_coords[k][1][1]])
        if k == 'main':
            stat = stat_converter(data)
            if assume_max_lv_gear is True: item['mainStat'] = [stat,max_stat(data,item)]
            else: item['mainStat'] = [stat,digit_filter(data)]     
        
        if k == 'subs':
            for n,entry in enumerate(data.split('\n')):
                if n > 3:
                    break
                stat = stat_converter(entry)
                val = digit_filter(entry.replace('T','7'))
                item['subStat'+str(n+1)] = [stat,val]
        if k == 'set':
            r = ''
            data = data.strip().replace(' ', '')
            if '치명' in data:
                r = 'Critical'
            elif '파멸' in data:
                r = 'Destruction'
            elif '흡혈' in data:
                r = 'Lifesteal'
            elif '공격' in data:
                r = 'Attack'
            elif '분노' in data:
                r = 'Rage'
            elif '속도' in data:
                r = 'Speed'
            elif '면역' in data:
                r = 'Immunity'
            elif '체력' in data:
                r = 'Health'
            elif '방어' in data:
                r = 'Defense'
            elif '저항' in data:
                r = 'Resist'
            elif '협동' in data:
                r = 'Unity'
            elif '반격' in data:
                r = 'Counter'
            elif '적중' in data:
                r = 'Hitrate'
            item['set'] = r
    
    # A common OCR error is to fail to read the level. If this happens, we need to set it to 1 so it will import properly.
    if item['level'] == 0: item['level'] = 1
    
    # Check to see if there are duplicates, and if not, give item unique ID and place in 'export'
    if item not in temp_list:
        temp_list.append(item)
        item['id'] = 'jt'+''.join(random.choice(digits+ascii_lowercase) for _ in range(6))
        export['items'].append(item)
    
    print(len(export['items']),"exported out of",len(filenames))
    
    
# Export dictionary to json for importing into optimizer
with open('exported_gear.json', 'w') as f: json.dump(export, f)

print('JSON file finished!')


# In[ ]:




